#### This notebook demonstrates the use of an odds-equalizing post-processing algorithm for bias mitigiation.


In [1]:
import aif360
import numpy as np
import pandas as pd

from aif360.metrics import BinaryLabelDatasetMetric
from aif360.metrics import ClassificationMetric
from aif360.metrics.utils import compute_boolean_conditioning_vector

from aif360.metrics import utils
from aif360.datasets import BinaryLabelDataset
from aif360.datasets.multiclass_label_dataset import MulticlassLabelDataset

from aif360.algorithms.preprocessing.optim_preproc_helpers.data_preproc_functions\
                import load_preproc_data_adult, load_preproc_data_compas


import sklearn 
from sklearn.linear_model import LogisticRegression
import imblearn
import matplotlib.pyplot as plt 
from sklearn.metrics import classification_report, confusion_matrix
import math 


In [2]:
german_data = pd.read_csv('german_data.csv')

In [3]:
sen_att= 'sex'

In [4]:
#split data in a test val and train set

from sklearn.model_selection import train_test_split
X = german_data.loc[:, german_data.columns != 'approval']
y = german_data.loc[:, german_data.columns == 'approval']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify = y, random_state=0)


In [5]:
#apply SMOTE to the trainingset
from imblearn.over_sampling import SMOTE
os = SMOTE(random_state=4)

In [6]:
columns = X_train.columns
X_train_balanced, y_train_balanced=os.fit_sample(X_train, y_train)

In [7]:
X_train_balanced

,stat_check_acc,duration_month,credit_history,purpose,credit_amount,Age Group,savings_bonds,employment_since,installment_in_percent,sex,debtors_guarant,residence_since,property,other_installment_plans,housing,nr_credits,job,nr_dependants,phone
0,2,24,3,1,1246,0,1,2,4,0,1,2,1,2,2,1,2,1,1
1,1,12,3,1,900,0,5,3,4,0,1,2,3,3,2,1,3,1,1
2,4,6,3,1,672,1,1,1,1,1,1,4,1,3,2,1,1,1,2
3,4,10,3,2,2848,0,2,3,1,0,2,2,1,3,2,1,3,2,1
4,4,48,5,10,7629,0,5,5,4,0,1,2,3,1,2,2,4,2,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1115,1,33,1,10,4266,0,1,2,1,0,1,4,1,3,1,2,2,1,1
1116,1,25,3,3,2373,0,1,1,1,0,1,1,1,3,1,1,3,1,1
1117,3,19,4,1,2760,0,1,3,2,0,1,2,2,1,1,1,2,1,1
1118,2,35,3,10,4466,0,1,3,2,0,1,2,1,2,2,2,4,1,2


In [8]:
y_train_balanced

,approval
0,0
1,0
2,1
3,1
4,1
...,...
1115,0
1116,0
1117,0
1118,0


In [9]:
balanced_german_train_df = pd.concat([X_train_balanced, y_train_balanced], axis=1, join="inner")

In [10]:
german_test_df = pd.concat([X_test, y_test], axis=1, join="inner")

In [11]:
from aif360.algorithms.preprocessing import DisparateImpactRemover
dataset_orig_train = aif360.datasets.BinaryLabelDataset(
    favorable_label=1,
    unfavorable_label=0,
    df=balanced_german_train_df,
    label_names=['approval'],
    protected_attribute_names=[sen_att])

In [12]:
from aif360.algorithms.preprocessing import DisparateImpactRemover
dataset_orig_test = aif360.datasets.BinaryLabelDataset(
    favorable_label=1,
    unfavorable_label=0,
    df=german_test_df,
    label_names=['approval'],
    protected_attribute_names=[sen_att])

In [13]:
from aif360.algorithms.preprocessing import DisparateImpactRemover
dataset_orig = aif360.datasets.BinaryLabelDataset(
    favorable_label=1,
    unfavorable_label=0,
    df=german_data,
    label_names=['approval'],
    protected_attribute_names=[sen_att])

In [14]:
privileged_groups = [{sen_att: 0}]
unprivileged_groups = [{sen_att: 1}]
cost_constraint = "fnr"
randseed = 12345679 

In [15]:
### logistic regression ### 

from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import roc_curve

# Placeholder for predicted and transformed datasets
dataset_orig_train_pred = dataset_orig_train.copy(deepcopy=True)
dataset_orig_test_pred = dataset_orig_test.copy(deepcopy=True)
dataset_new_test_pred = dataset_orig_test.copy(deepcopy=True)

# Logistic regression classifier and predictions for training data
scale_orig = StandardScaler()
X_train = scale_orig.fit_transform(dataset_orig_train.features)
y_train = dataset_orig_train.labels.ravel()
lmod = LogisticRegression()
lmod.fit(X_train, y_train)

fav_idx = np.where(lmod.classes_ == dataset_orig_train.favorable_label)[0][0]
y_train_pred_prob = lmod.predict_proba(X_train)[:,fav_idx]

X_test = scale_orig.transform(dataset_orig_test.features)
y_test_pred_prob = lmod.predict_proba(X_test)[:,fav_idx]

class_thresh = 0.5
dataset_orig_train_pred.scores = y_train_pred_prob.reshape(-1,1)

dataset_orig_test_pred.scores = y_test_pred_prob.reshape(-1,1)

y_train_pred = np.zeros_like(dataset_orig_train_pred.labels)
y_train_pred[y_train_pred_prob >= class_thresh] = dataset_orig_train_pred.favorable_label
y_train_pred[~(y_train_pred_prob >= class_thresh)] = dataset_orig_train_pred.unfavorable_label
dataset_orig_train_pred.labels = y_train_pred
    
y_test_pred = np.zeros_like(dataset_orig_test_pred.labels)
y_test_pred[y_test_pred_prob >= class_thresh] = dataset_orig_test_pred.favorable_label
y_test_pred[~(y_test_pred_prob >= class_thresh)] = dataset_orig_test_pred.unfavorable_label
dataset_orig_test_pred.labels = y_test_pred

In [16]:
# Metrics function
from collections import OrderedDict
from aif360.metrics import ClassificationMetric

def compute_metrics(dataset_true, dataset_pred, 
                    unprivileged_groups, privileged_groups,
                    disp = True):
    """ Compute the key metrics """
    classified_metric_pred = ClassificationMetric(dataset_true,
                                                 dataset_pred, 
                                                 unprivileged_groups=unprivileged_groups,
                                                 privileged_groups=privileged_groups)
    metrics = OrderedDict()
    metrics["Balanced accuracy"] = 0.5*(classified_metric_pred.true_positive_rate()+
                                             classified_metric_pred.true_negative_rate())
    metrics["Statistical parity difference"] = classified_metric_pred.statistical_parity_difference()
    metrics["Disparate impact"] = classified_metric_pred.disparate_impact()
    metrics["Average odds difference"] = classified_metric_pred.average_odds_difference()
    metrics["Equal opportunity difference"] = classified_metric_pred.equal_opportunity_difference()
    
    if disp:
        for k in metrics:
            print("%s = %.4f" % (k, metrics[k]))

In [17]:
### Equal odds Hardt et al.

from aif360.datasets import StandardDataset
from aif360.metrics import BinaryLabelDatasetMetric, ClassificationMetric, DatasetMetric 
from aif360.algorithms.preprocessing import LFR, Reweighing
from aif360.algorithms.inprocessing import AdversarialDebiasing, PrejudiceRemover
from aif360.algorithms.postprocessing import CalibratedEqOddsPostprocessing, EqOddsPostprocessing, RejectOptionClassification
from time import time

EOPP = EqOddsPostprocessing(privileged_groups = privileged_groups,
                             unprivileged_groups = unprivileged_groups,
                             seed=42)
EOPP = EOPP.fit(dataset_orig_test, dataset_orig_test_pred)
dataset_transf_test_pred = EOPP.predict(dataset_orig_test_pred)

In [18]:
 metric_test_aft = compute_metrics(dataset_orig_test, dataset_transf_test_pred, 
                                      unprivileged_groups, privileged_groups,
                                      disp = True)

Balanced accuracy = 0.7000
Statistical parity difference = -0.0690
Disparate impact = 0.9012
Average odds difference = -0.0058
Equal opportunity difference = -0.0555


In [19]:

y_test = dataset_orig_test.labels.ravel()
y_pred = dataset_transf_test_pred.labels.ravel()

In [20]:
### extract test labels and prediction label for traditional model evaluation ###

y_test = dataset_orig_test.labels.ravel()
y_pred = dataset_transf_test_pred.labels.ravel()

### obtain performance metrics###

matrix = sklearn.metrics.confusion_matrix(y_test, y_pred)
accuracy_score = sklearn.metrics.accuracy_score(y_test, y_pred)
print(matrix, accuracy_score)
print(classification_report(y_test, y_pred))

[[ 36  24]
 [ 28 112]] 0.74
              precision    recall  f1-score   support

         0.0       0.56      0.60      0.58        60
         1.0       0.82      0.80      0.81       140

    accuracy                           0.74       200
   macro avg       0.69      0.70      0.70       200
weighted avg       0.75      0.74      0.74       200

